<a href="https://colab.research.google.com/github/Chocoding1/Machine_Learning_Deep_Learning/blob/main/07_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **심층 신경망**

## 2개의 층

In [1]:
# 케라스 API를 사용하여 패션 MNIST 데이터셋 불러오기
from tensorflow import keras

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [3]:
# 데이터 전처리
from sklearn.model_selection import train_test_split

# 이미지의 픽셀값 0 ~ 1 사이로 변환
train_scaled = train_input / 255.0
# 2차원 배열 -> 1차원 배열
train_scaled = train_scaled.reshape(-1, 28*28)
# 훈련 세트, 검증 세트 분리
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

위의 인공 신경망 모델의 층을 2개 추가해보자.<br>

입력층과 출력층 사이에 있는 모든 층을 **은닉층(hidden layer)**라고 한다.<br>

은닉층에는 활성화 함수가 적용되는데, 출력층에서의 활성화 함수가 시그모이드 함수와 소프트맥스 함수로 한정되어 있는 것과 달리 은닉층의 활성화 함수는 비교적 자유롭다. 대표적으로 시그모이드 함수와 렐루(Relu) 함수 등을 사용한다.

note) 은닉층에 활성화 함수를 적용하는 이유<br>
선형적인 계산에 비선형성을 주기 위해서

note) 회귀를 위한 신경망의 출력층에서는 어떤 활성화 함수를 사용하나?<br>
위의 시그모이드와 소프트맥스는 분류를 위한 신경망의 출력층에서 사용한다.(클래스에 대한 확률을 출력하기 위해)<br>
그러나 회귀의 출력은 임의의 어떤 숫자이므로 활성화 함수를 적용할 필요가 없다. 즉 출력층의 선형 방정식의 계산을 그대로 출력한다. 이렇게 하려면 Dense 층의 activation 매개변수에 아무런 값을 지정하지 않으면 된다.

이제 시그모이드 활성화 함수를 사용한 은닉층과 소프트맥스 함수를 사용한 출력층을 만들어보자.

In [5]:
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
dense2 = keras.layers.Dense(10, activation='softmax')

dense1 : 은닉층 / 100개의 뉴런을 가진 밀집층(은닉층의 뉴런 개수를 정하는 데에는 특별한 기준이 없다. 경험이 필요 / 그러나 한 가지, 적어도 출력층의 뉴런보다는 많게 만들어야 한다.)<br>
dense2 : 출력층 / 10개의 클래스를 분류하므로 10개의 뉴런을 가진 밀집층

## 심층 신경망 만들기

Sequential 클래스의 객체를 만들 때 여러 개의 층을 추가하려면 각 층을 리스트로 만들어 전달해야 한다. 주의할 점은 출력층을 가장 마지막에 둬야 한다. 즉 해당 리스트는 가장 처음 등장하는 은닉층에서 마지막 출력층의 순서로 나열해야 한다.

In [6]:
model = keras.Sequential([dense1, dense2])

keras는 모델의 summary() 메서드를 호출하면 층에 대한 유용한 정보를 얻을 수 있다.

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 100)               78500     
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Output Shape의 첫 번째 자원은 샘플의 개수를 나타내는데, 샘플의 개수가 아직 정의되지 않았기 때문에 None이다. keras 모델의 fit() 메서드에 훈련 데이터를 주입하면 이 데이터를 한 번에 모두 사용하지 않고 미니배치 경사 하강법을 사용한다.<br>
keras의 기본 미니배치 크기는 32이다. 이 값은 fit() 메서드의 batch_size 매개변수로 바꿀 수 있다. 따라서 샘플 개수를 고정하지 않고 어떤 배치 크기에도 유연하게 대응할 수 있도록 None으로 설정한다.<br>

두 번째 자원인 100은 은닉층의 뉴런 개수를 100으로 지정했기 때문에 나온 당연한 결과이다. 즉 샘플마다 784개의 픽셀값이 은닉층을 통과하면서 100개의 특성으로 압축되었다.

마지막으로는 모델 파라미터 개수가 출력된다.<br>
78500 = 785(픽셀 수) x 100(가중치 = 뉴런 수) + 100(절편 = 뉴런 수)

summary() 메서드의 마지막에는 총 모델 파라미터 개수와 훈련되는 파라미터 개수가 동일하게 79,510으로 나온다. 은닉층과 출력층의 파라미터 개수를 합친 값이다. 그 아래 훈련되지 않는 파라미터(Non-trainable params)는 0으로 나오는데, 간혹 경사 하강법으로 훈련되지 않는 파라미터를 가진 층이 있다.

## 층을 추가하는 다른 방법

1. Sequential 클래스 생성자 안에 바로 입력<br>
dense1과 dense2처럼 밀집층 객체를 따로 저장하여 쓸 일이 없기 때문에 Sequential 클래스의 생성자 안에서 바로 Dense 클래스의 객체를 만드는 경우가 많다.<br>
이렇게 작업하면 추가되는 층을 한눈에 쉽게 알아보는 장점이 있다.

In [8]:
model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
    keras.layers.Dense(10, activation='softmax', name='output')
], name='패션 MNIST 모델')

위에서 모델의 이름과 층의 이름을 지정했는데, 모델의 이름과 달리 층의 이름은 반드시 영문이어야 한다.

In [10]:
# 모델과 층의 이름 확인
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


위의 방법이 편리하지만 층이 아주 많아지면 Sequential 클래스 생성자가 매우 길어지고, 또 조건에 따라 층을 추가할 수도 없다.

2. add() 메서드 사용<br>
Dense 클래스의 객체를 따로 변수에 담지 않고 바로 add() 메서드로 전달한다. 이 방법은 한눈에 추가되는 층을 볼 수 있고 프로그램 실행 시 동적으로 층을 선택하여 추가할 수 있다.

In [12]:
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

In [13]:
# 모델 정보 확인
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 100)               78500     
                                                                 
 dense_6 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# 모델 훈련
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5725 - accuracy: 0.8058
Epoch 2/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4123 - accuracy: 0.8515
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3769 - accuracy: 0.8640
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3542 - accuracy: 0.8719
Epoch 5/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3353 - accuracy: 0.8780
